In [1]:
import threading
import hashlib
import base64
import sys
import os
import random

NUM_CANDIDATOS = 2
NUM_URNAS = 4
NUM_VOTOS = 100

# Classe que representa uma urna eletrônica
class UrnaEletronica:
    def __init__(self, ident):
        self.ident = ident
        self.votos = [0] * NUM_CANDIDATOS

    def adicionar_voto(self, candidato):
        if candidato <= NUM_CANDIDATOS:
            self.votos[candidato - 1] += 1

    def __str__(self):
        votos_str = ", ".join([f"Candidato {i + 1}: {v} votos" for i, v in enumerate(self.votos)])
        return f"Urna {self.ident}: {votos_str}"

# Classe que representa o servidor de borda
class ServidorBorda:
    def __init__(self, urnas):
        self.urnas = urnas
        self.resultados_encriptados = []  # Lista para armazenar os resultados encriptados
        self.chave_encriptacao = None  # Variável para armazenar a chave de encriptação

    # Função para gerar partes aleatórias a partir do arquivo de audio
    def gerar_bytes_aleatorios(self, arquivo, tamanho_byte):
        tamanho_arquivo = os.path.getsize(arquivo)
        num_bytes = tamanho_arquivo // tamanho_byte

        indices = list(range(num_bytes))
        random.shuffle(indices)

        with open(arquivo, 'rb') as file:
            for indice in indices:
                file.seek(indice * tamanho_byte)
                byte = file.read(tamanho_byte)
                yield byte

    # Função para gerar a chave de encriptação a partir do arquivo de áudio
    def gerar_chave_encriptacao(self):
        try:
            with open('chave.wav', 'rb') as file:
                gerador_bytes = self.gerar_bytes_aleatorios('chave.wav', 16)

                hash_sha256 = hashlib.sha256()

                for byte in gerador_bytes:
                    hash_sha256.update(byte)

                chave_encriptacao = hash_sha256.digest()
                chave_encriptacao = base64.b64encode(chave_encriptacao)

                return chave_encriptacao
        except FileNotFoundError:
            print("Erro: Arquivo chave.wav não encontrado.", file=sys.stderr)
            sys.exit(1)
        except IOError:
            print("Erro ao ler o arquivo chave.wav.", file=sys.stderr)
            sys.exit(1)

    # Função para processar os dados da urna
    def processar_dados_urna(self):
        self.chave_encriptacao = self.gerar_chave_encriptacao()
        for urna in self.urnas:
            for _ in range(NUM_VOTOS):
                candidato = random.randint(1, NUM_CANDIDATOS)
                urna.adicionar_voto(candidato)
                candidato_encriptado = candidato ^ int.from_bytes(self.chave_encriptacao, 'big')
                self.resultados_encriptados.append(candidato_encriptado)

# Classe que representa o servidor na nuvem
class ServidorNuvem:
    # Simula o cálculo de complexidade O(M^2) na nuvem
    def calcular_complexidade_nuvem(self, resultados_encriptados, chave_desencriptacao):
        resultado_desencriptado = sum([resultado_encriptado ^ int.from_bytes(chave_desencriptacao, 'big') for resultado_encriptado in resultados_encriptados])
        resultado_encriptado_nuvem = resultado_desencriptado ^ int.from_bytes(chave_desencriptacao, 'big')
        return resultado_encriptado_nuvem, chave_desencriptacao, resultado_desencriptado

def main():
    urnas = [UrnaEletronica(i) for i in range(NUM_URNAS)]

    servidor_borda1 = ServidorBorda(urnas[:2])
    servidor_borda2 = ServidorBorda(urnas[2:])

    thread1 = threading.Thread(target=servidor_borda1.processar_dados_urna)
    thread2 = threading.Thread(target=servidor_borda2.processar_dados_urna)
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()

    resultados_encriptados_borda1 = servidor_borda1.resultados_encriptados
    resultados_encriptados_borda2 = servidor_borda2.resultados_encriptados

    chave_encriptacao = servidor_borda1.chave_encriptacao
    servidor_nuvem = ServidorNuvem()
    resultado_nuvem = servidor_nuvem.calcular_complexidade_nuvem(resultados_encriptados_borda1 + resultados_encriptados_borda2, chave_encriptacao)

    print("Resultado encriptado na nuvem: {}".format(resultado_nuvem))
    print("Chave de encriptação: {}".format(chave_encriptacao))
    print("Chave de desencriptação: {}".format(chave_encriptacao))

    for urna in urnas:
        print(urna)

if __name__ == '__main__':
    main()


Resultado encriptado na nuvem: (272094126712709070068405726749650863617334683654135881099719455469191485379900354203324204668884794688563566, b'E8k7yW8m05gvn/AgTPq3bpogK8nIob1XRGkcboGajrk=', 274574587605866604606016644469356325222702225310673554235794166559568925389603298619559721112249858654208595)
Chave de encriptação: b'E8k7yW8m05gvn/AgTPq3bpogK8nIob1XRGkcboGajrk='
Chave de desencriptação: b'E8k7yW8m05gvn/AgTPq3bpogK8nIob1XRGkcboGajrk='
Urna 0: Candidato 1: 53 votos, Candidato 2: 47 votos
Urna 1: Candidato 1: 56 votos, Candidato 2: 44 votos
Urna 2: Candidato 1: 52 votos, Candidato 2: 48 votos
Urna 3: Candidato 1: 44 votos, Candidato 2: 56 votos


## Comentários

A ideia é usar os dados de áudio como uma fonte de entropia para gerar uma chave criptográfica forte.

A aleatoriedade desse arquivo de áudio torna a chave de encriptação imprevisível, o que é uma propriedade desejável em criptografia.

A principal razão pela qual a fonte de áudio com aleatoriedade escolhida pode aumentar a complexidade computacional para alguém tentar quebrar a criptografia está relacionada à natureza do áudio. Arquivos de áudio contêm uma grande quantidade de dados, e a aleatoriedade presente nesses dados pode aumentar a dificuldade de encontrar padrões ou realizar análises estatísticas para quebrar a criptografia.

Para um adversário tentar quebrar a criptografia, ele precisaria analisar e processar todos os dados presentes no arquivo de áudio, aplicar algoritmos de criptoanálise e realizar tentativas exaustivas para encontrar falhas ou identificar informações úteis que possam ajudar na quebra da chave de encriptação. Esses processos são intensivos em termos de recursos computacionais e podem exigir uma quantidade significativa de tempo e poder computacional.